In [70]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings
warnings.filterwarnings("ignore")
dataview_folder = './JAQS_Data/sh300'
dv = DataView()
dv.load_dataview(dataview_folder)
mask = dv.get_ts('mask_index_member')
can_enter = dv.get_ts('can_enter')
can_exit = dv.get_ts('can_exit')

Dataview loaded successfully.


In [71]:
dv.add_formula("Divert", "Correlation(close_adj,volume, 10)", is_quarterly=False).head()

symbol,000001.SZ,000002.SZ,000008.SZ,000009.SZ,000027.SZ,000039.SZ,000046.SZ,000060.SZ,000061.SZ,000063.SZ,...,601992.SH,601997.SH,601998.SH,603000.SH,603160.SH,603799.SH,603833.SH,603858.SH,603885.SH,603993.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20170104,0.619153,0.423168,0.574381,0.655714,0.917101,0.727518,0.510402,-0.158148,0.925816,-0.019721,...,0.816361,0.360972,0.765654,0.580854,0.716348,0.375721,NaN,0.040894,0.831913,0.363563
20170105,0.542934,0.537910,0.292191,0.911676,0.896521,0.982302,0.104997,0.000939,0.905170,0.007650,...,0.806621,0.121072,0.600212,0.568013,0.774913,0.373882,NaN,0.181619,0.687895,0.377078
20170106,0.568943,0.662510,0.580083,0.918926,0.944568,0.982897,0.121979,0.047974,0.850987,-0.075553,...,0.777973,0.162739,0.539002,0.274686,0.828625,0.420310,NaN,-0.153705,0.365959,0.516858
20170109,0.646533,0.779836,0.766952,0.919424,0.953359,0.868713,0.087202,0.015089,0.922392,-0.844900,...,0.833345,0.321368,0.436899,0.309179,0.802357,0.348273,NaN,-0.610093,-0.349445,0.201511
20170110,0.433205,0.855911,0.754620,0.875597,0.888046,0.809758,0.162877,0.819062,0.945716,-0.694729,...,0.819063,0.259565,0.358994,0.294556,0.867364,0.200631,NaN,-0.778134,0.218639,0.442252


In [72]:
from jaqs_fxdayu.research.signaldigger import process

Open = dv.get_ts("open_adj")
High = dv.get_ts("high_adj")
Low = dv.get_ts("low_adj")
Close = dv.get_ts("close_adj")
trade_status = dv.get_ts('trade_status')
mask_sus = trade_status == u'停牌'
# 剔除掉停牌期的数据　再计算指标
open_masked = process._mask_df(Open,mask=mask_sus)
high_masked = process._mask_df(High,mask=mask_sus)
low_masked = process._mask_df(Low,mask=mask_sus)
close_masked = process._mask_df(Close,mask=mask_sus)

    顺势指标 
原理： 
    用目前股价的波动程度和常态分布范围比较，来得出超买或超卖的结论，用于捕捉趋势反转点。 
算法： 
　  典型价格与典型价格的N日移动平均的差除以N日内典型价格的平均绝对偏差。 
用法： 
    当CCI小于-100时为买入信号，CCI大于100时为卖出信号； 
    股价产生背离现象时，是一项明显的警告信号。 
参数： 
    N　设定计算移动平均的天数，一般为14。''' 

In [101]:
from jaqs_fxdayu.data import signal_function_mod as sfm
CCI30 = sfm.ta(ta_method='CCI',
             ta_column=0, 
             Open=open_masked, 
             High=high_masked, 
             Low=low_masked, 
             Close=close_masked,
             Volume=None,
              timeperiod=30)
CCI14=sfm.ta('CCI',Close=close_masked,Low=low_masked,High=high_masked,  timeperiod=14 )

In [103]:
dv.append_df(CCI30,'CCI30')
dv.append_df(CCI14,'CCI14')

In [110]:
# dv.add_formula("CCIindex","(CCI30<= -100)&&(CCI30>100)",is_quarterly=False, add_data=True)

In [111]:
import numpy as np

#定义信号过滤条件-非指数成分
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = df_index_member == 0
    return mask_index_member

# 定义可买卖条件——未停牌、未涨跌停
def limit_up_down():
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False, add_data=True)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False, add_data=True)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

mask = mask_index_member()
can_enter,can_exit = limit_up_down()

In [113]:
from jaqs_fxdayu.research import SignalDigger
obj = SignalDigger(output_folder='./output',
                   output_format='pdf')

# 和处理因子的步骤一样 n_quantiles=1
# 不传入benchmark 可以分析绝对收益
obj.process_signal_before_analysis(signal=dv.get_ts("CCIindex"),
                                   price=dv.get_ts("close_adj"),
                                   high=dv.get_ts("high_adj"), # 可为空
                                   low=dv.get_ts("low_adj"),# 可为空
                                   n_quantiles=1,# quantile分类数
                                   mask=mask,# 过滤条件
                                   can_enter=can_enter,# 是否能进场
                                   can_exit=can_exit,# 是否能出场
                                   period=30,# 持有期
                                   # benchmark_price=dv.data_benchmark, # 基准价格 可不传入，持有期收益（return）计算为绝对收益
                                   commission = 0.0008,
                                   )
signal_data = obj.signal_data
signal_data.head()

Nan Data Count (should be zero) : 0;  Percentage of effective data: 66%


signal    return  upside_ret  downside_ret  quantile
trade_date symbol                                                         
20170222   000001.SZ     0.0 -0.039462   -0.000800     -0.054092         1
           000008.SZ     0.0  0.033369    0.061842     -0.045219         1
           000009.SZ     0.0 -0.053592    0.020520     -0.073896         1
           000027.SZ     0.0  0.004787    0.025736     -0.049683         1
           000039.SZ     0.0 -0.048364   -0.001946     -0.106817         1

In [115]:
from jaqs_fxdayu.research.signaldigger.analysis import analysis
result = analysis(signal_data, is_event=True, period=30)
print("——选股收益分析——")
print(result["ret"])
print("——最大潜在盈利/亏损分析——")
print(result["space"])

——选股收益分析——
           all_sample_ret
t-stat          16.414120
p-value          0.000000
skewness         0.951501
kurtosis         4.426096
Ann. Ret         0.053604
Ann. Vol         0.284263
Ann. IR          0.188570
occurance    61121.000000
——最大潜在盈利/亏损分析——
               all_sample_space
Up_sp Mean             0.070259
Up_sp Std              0.125867
Up_sp IR               0.558198
Up_sp Pct5             0.000000
Up_sp Pct25            0.009101
Up_sp Pct50            0.047487
Up_sp Pct75            0.109867
Up_sp Pct95            0.251296
Up_sp Occur        61121.000000
Down_sp Mean          -0.100495
Down_sp Std            0.202404
Down_sp IR            -0.496507
Down_sp Pct5          -0.268679
Down_sp Pct25         -0.097883
Down_sp Pct50         -0.045672
Down_sp Pct75         -0.013425
Down_sp Pct95          0.000000
Down_sp Occur      61121.000000


In [118]:
# 可视化
import matplotlib.pyplot as plt
#obj.create_full_report()
#plt.show()

In [126]:
from jaqs_fxdayu.research.signaldigger import multi_factor

ic = dict()
factors_dict = {signal:dv.get_ts(signal) for signal in ["pb","CCIindex"] }
for period in [5, 15, 30]:
    ic[period]=multi_factor.get_factors_ic_df(factors_dict,
                                              price=dv.get_ts("close_adj"),
                                              high=dv.get_ts("high_adj"), # 可为空
                                              low=dv.get_ts("low_adj"),# 可为空
                                              n_quantiles=5,# quantile分类数
                                              mask=mask,# 过滤条件
                                              can_enter=can_enter,# 是否能进场
                                              can_exit=can_exit,# 是否能出场
                                              period=period,# 持有期
                                              benchmark_price=dv.data_benchmark, # 基准价格 可不传入，持有期收益（return）计算为绝对收益
                                              commission = 0.0008,
                                              )

Nan Data Count (should be zero) : 0;  Percentage of effective data: 76%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 66%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 76%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 66%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 76%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 66%
